In [0]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf



In [0]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28
log_filepath = './log'
!rm -rf ./log

In [37]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices ワンホットエンコーディング
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

old_session = KTF.get_session()



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [38]:
with tf.Graph().as_default(): #withでくくる必要ないっぽい？
  session = tf.Session('')
  KTF.set_session(session)
  KTF.set_learning_phase(1)
  #denseは名前つけたほうがわかりやすいかも
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', name = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax', name = 'softmax'))
  model.summary()

  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
  
  tb_cb = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)
  cbks = [tb_cb]

  history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks = cbks,
          verbose=1,
          validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  
KTF.set_session(old_session)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
relu (Dense)                 (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

60000/60000 [==============================] - 163s 3ms/step - loss: 0.2729 - acc: 0.9148 - val_loss: 0.0985 - val_acc: 0.9675
Epoch 2/12
37504/60000 [=================>............] - ETA: 59s - loss: 0.0906 - acc: 0.9729

60000/60000 [==============================] - 166s 3ms/step - loss: 0.0884 - acc: 0.9738 - val_loss: 0.0733 - val_acc: 0.9758
Epoch 3/12
48768/60000 [=======================>......] - ETA: 30s - loss: 0.0679 - acc: 0.9798

60000/60000 [==============================] - 167s 3ms/step - loss: 0.0666 - acc: 0.9800 - val_loss: 0.0652 - val_acc: 0.9800
Epoch 4/12
52992/60000 [=========================>....] - ETA: 18s - loss: 0.0573 - acc: 0.9828

60000/60000 [==============================] - 166s 3ms/step - loss: 0.0568 - acc: 0.9829 - val_loss: 0.0538 - val_acc: 0.9821
Epoch 5/12
54656/60000 [==========================>...] - ETA: 13s - loss: 0.0478 - acc: 0.9849

...

60000/60000 [==============================] - 164s 3ms/step - loss: 0.0350 - acc: 0.9894 - val_loss: 0.0463 - val_acc: 0.9870
Epoch 9/12
55552/60000 [==========================>...] - ETA: 11s - loss: 0.0314 - acc: 0.9900

60000/60000 [==============================] - 164s 3ms/step - loss: 0.0317 - acc: 0.9900 - val_loss: 0.0533 - val_acc: 0.9841
Epoch 10/12
55296/60000 [==========================>...] - ETA: 12s - loss: 0.0296 - acc: 0.9906

60000/60000 [==============================] - 164s 3ms/step - loss: 0.0298 - acc: 0.9905 - val_loss: 0.0558 - val_acc: 0.9841
Epoch 11/12
55296/60000 [==========================>...] - ETA: 12s - loss: 0.0277 - acc: 0.9914

60000/60000 [==============================] - 164s 3ms/step - loss: 0.0283 - acc: 0.9912 - val_loss: 0.0510 - val_acc: 0.9865
Epoch 12/12
55296/60000 [==========================>...] - ETA: 12s - loss: 0.0261 - acc: 0.9920

60000/60000 [==============================] - 164s 3ms/step - loss: 0.0262 - acc: 0.9919 - val_loss: 0.0533 - val_acc: 0.9864
Test loss: 0.0529120142178057
Test accuracy: 0.9845


In [39]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
#下2行はインストールしてなければ実行
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://a6783d02.ngrok.io


In [40]:
!ls ./log -c

events.out.tfevents.1524030687.c2351a6048b6
